# Exploring the Impact of Structured Representations in Scenario Generation Based on Large Language Models

## Preparation

In [ ]:
import os
os.chdir('/content')

# fetch codebase
CODE_DIR = 'lctgen'
os.makedirs(f'./{CODE_DIR}', exist_ok=True)
!git clone https://github.com/Ariostgx/lctgen.git $CODE_DIR
os.chdir('/content/lctgen')

In [ ]:
%pip install -q -r requirements_colab.txt --quiet
%pip install google-generativeai --quiet
%pip install ipympl --quiet

In [ ]:
!gdown https://drive.google.com/uc?id=17_TI-q4qkCOt988spWIZCqDLkZpMSptO -O data.zip
!unzip data.zip -d /content/lctgen/data/demo/waymo/

In [ ]:
!gdown https://drive.google.com/uc?id=1_s_35QO6OiHHgDxHHAa7Djadm-_I7Usr -O example.ckpt
!mkdir /content/lctgen/checkpoints
!mv example.ckpt /content/lctgen/checkpoints

In [ ]:
from google.colab import output

output.enable_custom_widget_manager()

## Setup

In [ ]:
import itertools
import json
import math
import sys

import torch
import ipywidgets

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import openai
import google.generativeai as genai

from google.colab import userdata
from datetime import datetime
from IPython.display import HTML, Javascript
from matplotlib.patches import Wedge
from pprint import pprint
from shapely.geometry import Point, Polygon, LineString

# LCTGen
from lctgen.config.default import get_config
from lctgen.core.registry import registry
from lctgen.datasets.waymo_open_motion import WaymoOpenMotionDataset
from lctgen.inference.utils import output_formating_cot, map_retrival, get_map_data_batch, load_all_map_vectors
from lctgen.models.utils import visualize_input_seq, visualize_output_seq, transform_traj_output_to_waymo_agent

# TrafficGen
from trafficgen.utils.typedef import *
from trafficgen.utils.data_process.agent_process import WaymoAgent

In [ ]:
in_colab = "google.colab" in sys.modules

In [ ]:
cfg_file = 'cfgs/demo_inference.yaml'
cfg = get_config(cfg_file)

model_cls = registry.get_model(cfg.MODEL.TYPE)
model = model_cls.load_from_checkpoint(cfg.LOAD_CHECKPOINT_PATH, config=cfg, metrics=[], strict=False)
model.eval()

map_data_file = 'data/demo/waymo/demo_map_vec.npy'
map_vecs, map_ids = load_all_map_vectors(map_data_file)

## Inference

### OpenAI API

In [ ]:
openai.organization = userdata.get("OPENAI_ORGANIZATION")
openai.api_key = userdata.get("OPENAI_API_KEY")

In [ ]:
llm_model_name = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4"]

llm_cfg = get_config("/content/lctgen/lctgen/gpt/cfgs/attr_ind_motion/non_api_cot_attr_20m.yaml")
llm_cfg.merge_from_list(["LLM.CODEX.MODEL", llm_model_name])

llm_model = registry.get_llm('codex')(llm_cfg)

In [ ]:
query = 'V1 speeds up and crashes V2 from back'  # @param {type:"string"}
llm_result = llm_model.forward(query)

print('LLM inference result:')
print(llm_result)

### Google AI Studio API

In [ ]:
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

for llm_model in genai.list_models():
    pprint(llm_model)

In [ ]:
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

# Set up the model
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]

llm_model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest", generation_config=generation_config, safety_settings=safety_settings)

In [ ]:
query = 'V1 goes straight and collides with V2 while V2 turns left'  # @param {type:"string"}


prompt = "## INSTRUCTIONS\n"

with open("/content/lctgen/lctgen/gpt/prompts/attr_ind_motion/sys_non_api_cot_attr_20m.prompt") as fp:
    prompt += fp.read()

prompt += "\n---\n## QUERIES\n"

with open("/content/lctgen/lctgen/gpt/prompts/attr_ind_motion/non_api_cot_attr_20m.prompt") as fp:
    prompt += fp.read()

prompt = prompt.replace("INSERT_QUERY_HERE", query)

print("Prompt:")
print(prompt)
print("\n\n\n")


result = llm_model.generate_content(prompt)
llm_result = result.text


print("LLM inference result:")
print(llm_result)

### Predefined Result

In [ ]:
llm_result = """
Actor Vector:
- 'V1': [-1, 0, 0, 6, 4, 3, 3, 3]
- 'V2': [0, 0, 1, 1, 1, 1, 1, 1]
- 'V3': [2, 0, 0, 2, 4, 4, 3, 3]
- 'V4': [1, 0, 1, 2, 4, 4, 3, 3]
- 'V5': [0, 1, 2, 0, 0, 0, 0, 0]
- 'V6': [0, 1, 2, 0, 0, 0, 0, 0]
- 'V7': [0, 1, 2, 0, 0, 0, 0, 0]
- 'V8': [0, 1, 2, 0, 0, 0, 0, 0]
- 'V9': [0, 1, 1, 2, 4, 4, 4, 4]
- 'V10': [0, 1, 1, 2, 4, 4, 4, 4]
- 'V11': [0, 1, 1, 1, 4, 4, 4, 4]
- 'V12': [3, 1, 0, 2, 4, 4, 4, 4]
Map Vector:
- 'Map': [2, 2, 2, 2, 1, 2]
"""